## Mini Project 03- NLP Emotions: Text Features

Dataset:
- A. Tripathi, "Emotion Classification NLP", Kaggle.com, 2021. [Online]. Available: https://www.kaggle.com/datasets/anjaneyatripathi/emotion-classification-nlp. [Accessed: 16- Jul- 2022].

Sources:
- WASSA-2017 Shared Task on Emotion Intensity. Saif M. Mohammad and Felipe Bravo-Marquez. In Proceedings of the EMNLP 2017 Workshop on Computational Approaches to Subjectivity, Sentiment, and Social Media (WASSA), September 2017, Copenhagen, Denmark.
BibTex

Emotion Labels:
- joy: 1
- sadness: 2
- anger: 3
- fear: 4

### Import Libraries

In [1]:
## Import Libraries
import numpy as np
import pandas as pd
import re as regex
import spacy
from pathlib import Path
import time


import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import plot_confusion_matrix
from simple_colors import *

import string
from collections import Counter
import re as regex
from sklearn import metrics
from sklearn.metrics import classification_report

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

import neattext.functions as nfx
import nltk

import warnings
warnings.filterwarnings('ignore')

import import_ipynb
import nlpEmotion_functions
from nlpEmotion_functions import show_summary_report, calcRocPrecisionRecall, testCalcRocPrecisionRecall, calcRocAuc
from sklearn.decomposition import PCA
from sklearn.decomposition import LatentDirichletAllocation

%matplotlib inline

importing Jupyter notebook from nlpEmotion_functions.ipynb


### Load Data

In [2]:
dfNlpFeatures = pd.read_csv("nlpTrainFeatures.csv")
dfNlpFeatures.head()

,Unnamed: 0,text,label,short,char_count,word_count,word_density,adj_count,adv_count,noun_count,num_count,pron_count,propn_count,verb_count
0,0,Just got back from seeing @GaryDelaney in Burs...,1,get see burslem amaze face hurt laugh hilarious,110,17,6.470588,2,5,5,0,0,2,5
1,1,Oh dear an evening of absolute hilarity I don'...,1,dear even absolute hilarity don think laugh lo...,94,20,4.700000,2,2,2,0,2,0,2
2,2,Been waiting all week for this game ❤️❤️❤️ #ch...,1,wait week game red heart red heart red heart c...,60,11,5.454545,1,0,0,0,0,1,1
3,3,"@gardiner_love : Thank you so much, Gloria! Yo...",1,thank gloria sweet thoughtful day joyful love ...,129,24,5.375000,3,6,6,0,6,1,3
4,4,I feel so blessed to work with the family that...,1,feel bless work family nanny red heart love am...,114,22,5.181818,2,1,1,0,5,2,6


In [3]:
dfNlpFeatures.drop(['Unnamed: 0'], axis= 1, inplace= True)

In [4]:
dfNlpFeatures.head()

,text,label,short,char_count,word_count,word_density,adj_count,adv_count,noun_count,num_count,pron_count,propn_count,verb_count
0,Just got back from seeing @GaryDelaney in Burs...,1,get see burslem amaze face hurt laugh hilarious,110,17,6.470588,2,5,5,0,0,2,5
1,Oh dear an evening of absolute hilarity I don'...,1,dear even absolute hilarity don think laugh lo...,94,20,4.700000,2,2,2,0,2,0,2
2,Been waiting all week for this game ❤️❤️❤️ #ch...,1,wait week game red heart red heart red heart c...,60,11,5.454545,1,0,0,0,0,1,1
3,"@gardiner_love : Thank you so much, Gloria! Yo...",1,thank gloria sweet thoughtful day joyful love ...,129,24,5.375000,3,6,6,0,6,1,3
4,I feel so blessed to work with the family that...,1,feel bless work family nanny red heart love am...,114,22,5.181818,2,1,1,0,5,2,6


In [5]:
X= dfNlpFeatures.iloc[:, 2:]
X[0:10]

,short,char_count,word_count,word_density,adj_count,adv_count,noun_count,num_count,pron_count,propn_count,verb_count
0,get see burslem amaze face hurt laugh hilarious,110,17,6.470588,2,5,5,0,0,2,5
1,dear even absolute hilarity don think laugh lo...,94,20,4.700000,2,2,2,0,2,0,2
2,wait week game red heart red heart red heart c...,60,11,5.454545,1,0,0,0,0,1,1
3,thank gloria sweet thoughtful day joyful love ...,129,24,5.375000,3,6,6,0,6,1,3
4,feel bless work family nanny red heart love am...,114,22,5.181818,2,1,1,0,5,2,6
5,today reach subscriber goodday thankful,62,10,6.200000,1,0,0,1,1,1,1
6,good morning love happy day fall let awesome a...,128,17,7.529412,5,0,0,0,1,0,3
7,bridgetjonesbaby best thing see age funny miss...,102,16,6.375000,2,1,1,0,2,3,3
8,get see burslem amaze face hurt laugh,100,16,6.250000,1,5,5,0,0,2,5
9,think holiday cheerful meet thenicebot,95,17,5.588235,1,2,2,0,5,0,3


In [6]:
X.shape

(7102, 11)

In [7]:
y= dfNlpFeatures[dfNlpFeatures.columns[1]]
y[0:10]

0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
Name: label, dtype: int64

In [8]:
y.shape

(7102,)

In [9]:
# split the dataset
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 42)

In [10]:
X_train.head()

,short,char_count,word_count,word_density,adj_count,adv_count,noun_count,num_count,pron_count,propn_count,verb_count
181,big change pray think deserve positivity optimism,87,17,5.117647,2,0,0,0,6,0,3
4793,majority client family partner day job life wr...,121,19,6.368421,5,0,0,0,2,1,3
794,don know worst purpose,63,12,5.250000,1,0,0,0,2,2,1
3593,amazon prime like help girl lol,73,15,4.866667,1,1,1,0,2,2,3
1536,say teufel reassign organization year wish soo...,139,21,6.619048,3,0,0,1,2,1,4


In [11]:
X_train.shape

(5681, 11)

In [12]:
X_trainVect= X_train['short']
X_testVect= X_test['short']

In [13]:
X_trainVect.shape

(5681,)

In [14]:
# create a count vectorizer object
count_vect = CountVectorizer(analyzer= 'word',
                             token_pattern = r'\w{1,}')

# Learn a vocabulary dictionary of all tokens in the raw documents
# count_vect.fit(X2)

# Transform documents to document-term matrix.
X_train_count = count_vect.fit_transform(X_trainVect.astype('str'))
X_test_count = count_vect.transform(X_testVect.astype('str'))

In [15]:
X_train_count.shape

(5681, 8917)

In [16]:
X_test_count.shape

(1421, 8917)

In [17]:
X_train_count

<5681x8917 sparse matrix of type '<class 'numpy.int64'>'
	with 43027 stored elements in Compressed Sparse Row format>

In [18]:
dfTrainCount= pd.DataFrame(X_train_count.toarray(), columns= count_vect.get_feature_names_out())
dfTestCount= pd.DataFrame(X_test_count.toarray(), columns= count_vect.get_feature_names_out())

In [19]:
dfTrainCount.head()

,aaa,aaaaarrrrgghhh,aaahhhh,aameen,aaron,aarp,aateam,abandonment,abby,abc,...,zionist,zipper,zombie,zone,zoolander,zumba,деньги,الخفجي,الوطني,اليوم
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
dfTrainCount.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5681 entries, 0 to 5680
Columns: 8917 entries, aaa to اليوم
dtypes: int64(8917)
memory usage: 386.5 MB


In [21]:
%%time
# Apply dimensionality reduction to Xs using transform
pca02= PCA(n_components= 800)
# Fit Xs
pcaModel02_Xtrain= pca02.fit_transform(dfTrainCount)
pcaModel02_Xtest= pca02.transform(dfTestCount)
exp_var_pca02= pca02.explained_variance_ratio_
cum_sum_eigenvalues02 = np.cumsum(exp_var_pca02)
pca02.singular_values_

CPU times: total: 1min 13s
Wall time: 12.1 s


array([73.93969208, 50.27948845, 29.97062224, 23.67411587, 22.19651456,
       20.43183976, 19.96653746, 19.77802988, 18.38179949, 18.02188342,
       17.40766825, 16.42026227, 16.25200554, 16.04915613, 15.78202582,
       15.57249489, 15.43370861, 15.40186909, 15.02722682, 14.88185134,
       14.75278026, 14.47865445, 14.15210691, 13.89613836, 13.26276417,
       12.99838689, 12.96867321, 12.8253393 , 12.69445881, 12.67862822,
       12.50372389, 12.38490866, 12.34889783, 12.24245511, 12.167295  ,
       12.10895177, 11.98215536, 11.8319117 , 11.80161411, 11.67773698,
       11.66206046, 11.60116401, 11.58196816, 11.42895628, 11.37976078,
       11.32887857, 11.2843908 , 11.16654744, 11.08567587, 11.00792749,
       10.9681238 , 10.87534683, 10.77738636, 10.72658831, 10.65828387,
       10.5453552 , 10.49091679, 10.44817946, 10.39705812, 10.37210137,
       10.29913924, 10.27320707, 10.22775753, 10.21997041, 10.12041349,
       10.08153376, 10.01834928,  9.98715787,  9.96987759,  9.93

In [22]:
np.sum(exp_var_pca02)

0.7679501716973767

In [23]:
pca_dfTrain= pd.DataFrame(pcaModel02_Xtrain)
pca_dfTest= pd.DataFrame(pcaModel02_Xtest)
pca_dfTrain.head()

,0,1,2,3,4,5,6,7,8,9,...,790,791,792,793,794,795,796,797,798,799
0,-0.009341,-0.161966,0.042974,-0.005137,-0.043912,-0.046542,0.008988,-0.036498,0.050643,-0.054951,...,0.064426,-0.057251,0.016699,0.007242,-0.047114,-0.030034,-0.010006,0.046275,-0.017626,-0.056490
1,-0.023999,-0.163796,-0.022175,0.049369,-0.070448,0.102078,-0.011344,0.055404,-0.335255,0.085782,...,0.113133,0.073304,0.022938,0.019587,-0.050848,-0.037764,0.039547,-0.044137,-0.014833,-0.027505
2,-0.023089,-0.175600,0.053482,0.017146,-0.002203,-0.716857,0.812790,-0.337689,0.027861,-0.057376,...,0.077665,-0.026029,-0.016435,0.004488,-0.022869,-0.002134,0.010364,-0.017779,0.033141,-0.003392
3,-0.023156,-0.165316,0.039625,-0.023168,0.037558,-0.572211,-0.408871,0.558861,0.125535,-0.044002,...,0.016875,0.000689,0.004664,-0.012232,0.011492,-0.010696,-0.003352,0.008569,0.027213,0.002218
4,-0.023021,-0.179607,0.038585,-0.004142,-0.041601,0.048619,-0.121212,0.032010,-0.002069,-0.020522,...,0.021699,-0.105922,0.042257,-0.059504,0.002873,0.041124,-0.005887,-0.004106,-0.016797,-0.029534


In [24]:
%%time
# train a LDA Model
lda_model = LatentDirichletAllocation(n_components = 20, learning_method = 'online', max_iter = 20)

Xtrain_topics = lda_model.fit_transform(X_train_count)
Xtest_topics = lda_model.transform(X_test_count)
topic_word = lda_model.components_ 
vocab = count_vect.get_feature_names_out()

CPU times: total: 18.1 s
Wall time: 18.1 s


In [25]:
Xtrain_topics.shape

(5681, 20)

In [26]:
Xtest_topics.shape

(1421, 20)

In [27]:
dfTrainTopic= pd.DataFrame(Xtrain_topics)
dfTestTopic= pd.DataFrame(Xtest_topics)

In [28]:
dfTrainTopic.shape

(5681, 20)

In [29]:
dfTestTopic.shape

(1421, 20)

In [30]:
X_train.reset_index(inplace= True)
X_test.reset_index(inplace= True)

In [31]:
X_train.head()

,index,short,char_count,word_count,word_density,adj_count,adv_count,noun_count,num_count,pron_count,propn_count,verb_count
0,181,big change pray think deserve positivity optimism,87,17,5.117647,2,0,0,0,6,0,3
1,4793,majority client family partner day job life wr...,121,19,6.368421,5,0,0,0,2,1,3
2,794,don know worst purpose,63,12,5.250000,1,0,0,0,2,2,1
3,3593,amazon prime like help girl lol,73,15,4.866667,1,1,1,0,2,2,3
4,1536,say teufel reassign organization year wish soo...,139,21,6.619048,3,0,0,1,2,1,4


In [32]:
X_test.head()

,index,short,char_count,word_count,word_density,adj_count,adv_count,noun_count,num_count,pron_count,propn_count,verb_count
0,6095,imagine celtic burst net wae attempt nutt old ...,88,17,5.176471,6,0,0,0,1,1,3
1,4447,need cheer,35,6,5.833333,0,0,0,0,1,0,2
2,4677,horrible person gag people quote,75,15,5.000000,1,1,1,0,4,0,3
3,4832,hand deck post msm trump start lead fla,122,24,5.083333,0,0,0,0,0,6,2
4,3796,gahh queue minute sport dept cancel queue,108,20,5.400000,0,3,3,1,0,4,2


In [33]:
X_train.drop(['index', 'short'], axis=1, inplace=True)
X_test.drop(['index', 'short'], axis=1, inplace=True)

In [34]:
pca_dfTrain.shape

(5681, 800)

In [35]:
pca_dfTest.shape

(1421, 800)

In [36]:
X_train.shape

(5681, 10)

In [37]:
X_test.shape

(1421, 10)

In [38]:
dfTrainCount.shape

(5681, 8917)

In [39]:
new_Xtrain= pd.concat([pca_dfTrain, dfTrainTopic, X_train], axis= 1)
new_Xtest= pd.concat([pca_dfTest, dfTestTopic, X_test], axis= 1)
# new_Xtrain = pd.merge(dfTrainCount, X_train, left_index=True, right_index=True)
topic_Xtrain= pd.concat([dfTrainTopic, X_train], axis= 1)
topic_Xtest= pd.concat([dfTestTopic, X_test], axis= 1)

In [40]:
new_Xtrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5681 entries, 0 to 5680
Columns: 830 entries, 0 to verb_count
dtypes: float64(821), int64(9)
memory usage: 36.0 MB


In [41]:
new_Xtrain.shape

(5681, 830)

In [42]:
new_Xtest.shape

(1421, 830)

In [43]:
topic_Xtrain.shape

(5681, 30)

In [44]:
topic_Xtest.shape

(1421, 30)

In [45]:
new_Xtrain.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,char_count,word_count,word_density,adj_count,adv_count,noun_count,num_count,pron_count,propn_count,verb_count
1151,-0.022710,-0.174128,0.047597,-0.010820,-0.090968,0.043056,0.049947,-0.078646,-0.035753,-0.016584,...,59,12,4.916667,1,0,0,0,1,4,1
2955,-0.022883,-0.153794,0.055800,0.003248,-0.071164,0.081836,-0.056010,-0.007647,-0.011241,-0.023975,...,55,8,6.875000,1,1,1,0,2,0,2
1088,-0.023234,-0.167383,0.059215,-0.038829,0.077836,-0.573941,-0.424952,0.582582,0.104448,-0.054113,...,52,9,5.777778,0,2,2,0,3,0,2
1952,-0.022217,-0.171742,0.034522,0.001009,-0.058937,0.089791,-0.042548,-0.046148,-0.009970,-0.008333,...,132,19,6.947368,0,0,0,0,1,3,3
703,-0.023578,-0.170092,0.032002,-0.024018,-0.051153,0.051579,-0.064175,-0.013520,-0.052386,0.001611,...,118,20,5.900000,5,1,1,0,1,1,2


In [46]:
new_Xtest.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,char_count,word_count,word_density,adj_count,adv_count,noun_count,num_count,pron_count,propn_count,verb_count
381,-0.023471,-0.176481,0.044219,-0.004873,-0.060616,0.047007,-0.031070,-0.036961,0.019822,0.042344,...,139,22,6.318182,3,1,1,0,1,2,2
217,-0.023397,-0.176177,0.032350,0.039637,0.002085,-0.008300,-0.141123,0.029271,0.027653,0.013784,...,128,21,6.095238,1,0,0,0,2,4,6
780,-0.022555,-0.168478,0.043821,0.005467,-0.057119,0.092911,-0.061036,-0.025767,-0.017285,0.002917,...,81,19,4.263158,1,1,1,1,3,0,2
488,-0.022308,-0.169907,0.039725,-0.006996,-0.069634,0.096029,-0.055662,-0.027708,0.014701,0.004936,...,68,14,4.857143,0,0,0,1,3,0,1
60,-0.022035,-0.171530,0.034988,-0.007809,-0.073709,0.090639,-0.039380,-0.033065,-0.012470,-0.012993,...,10,2,5.000000,1,0,0,0,0,0,0


In [47]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)

    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)

    return accuracy_score(predictions, y_test)

In [48]:
%%time
# Naive Bayes on Count Vectors
accuracy1 = train_model(LinearSVC(), X_train_count, y_train, X_test_count)
print('NB, Count Vectors    : %.4f\n' % accuracy1)

NB, Count Vectors    : 0.8360

CPU times: total: 422 ms
Wall time: 422 ms


In [49]:
%%time
# Naive Bayes on Count Vectors
accuracy2 = train_model(LinearSVC(), new_Xtrain, y_train, new_Xtest)
print('NB, Count Vectors    : %.4f\n' % accuracy2)

NB, Count Vectors    : 0.8149

CPU times: total: 39.2 s
Wall time: 39.2 s


In [50]:
%%time
# Naive Bayes on Count Vectors
accuracy3 = train_model(LinearSVC(), topic_Xtrain, y_train, topic_Xtest)
print('NB, Count Vectors    : %.4f\n' % accuracy3)

NB, Count Vectors    : 0.3251

CPU times: total: 2.27 s
Wall time: 2.27 s


Implementing PCA, adding text features, and using topic words does not improve the accuracy of the model and the runtime is increased. Therefore, these methods will be discarded when comparing the models